In [1]:
import numpy as np
import pandas as pd 
import os
from pathlib import Path
import nibabel as nib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tarfile

In [2]:
! pip install gdown

In [3]:
import gdown

url = 'https://drive.google.com/file/d/1zix-XyFhYTjlW4CnAnVrC7g0PvcEAbyi/view?usp=drivesdk'
file_id = url.split('/')[-2]
direct_download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

output = 'heart.tar'
gdown.download(direct_download_url, output, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1zix-XyFhYTjlW4CnAnVrC7g0PvcEAbyi&export=download
From (redirected): https://drive.google.com/uc?id=1zix-XyFhYTjlW4CnAnVrC7g0PvcEAbyi&export=download&confirm=t&uuid=e1f94a5f-b7ba-4ef0-9fec-3322362cafea
To: /kaggle/working/heart.tar
100%|██████████| 456M/456M [00:01<00:00, 237MB/s]


'heart.tar'

In [4]:
tar_path = "/kaggle/working/heart.tar"

with tarfile.open(tar_path, 'r') as tar:
    tar.extractall('./data')

In [5]:
root = Path("./data/Task02_Heart/imagesTr")
label = Path("./data/Task02_Heart/labelsTr")

In [6]:
def change_img_to_label_path(path):
    parts = list(path.parts)
    parts[parts.index('imagesTr')] = 'labelsTr'
    return Path(*parts)

In [7]:
sample_path = list(root.glob("la*"))[0]
sample_path_label = change_img_to_label_path(sample_path)

In [8]:
sample_path, sample_path_label

(PosixPath('data/Task02_Heart/imagesTr/la_018.nii.gz'),
 PosixPath('data/Task02_Heart/labelsTr/la_018.nii.gz'))

In [9]:
data = nib.load(sample_path)
label = nib.load(sample_path_label)

mri = data.get_fdata()
mask = label.get_fdata().astype(np.uint8)

In [10]:
# checking the orienation of scans
nib.aff2axcodes(data.affine)

('R', 'A', 'S')

In [11]:
! pip install celluloid

In [12]:
from celluloid import Camera
from IPython.display import HTML

fig = plt.figure()

camera = Camera(fig)

for i in range(mri.shape[2]):
    plt.imshow(mri[:, :, i], cmap='bone')
    mask_ = np.ma.masked_where(mask[:, :, i]==0, mask[:, :, i])
    plt.imshow(mask_, alpha=0.5)
    camera.snap()

# animation = camera.animate()
# HTML(animation.to_jshtml())

In [13]:
def normalize(full_volume):
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized):
    standardized = (normalized - normalized.min()) / (normalized.max() - normalized.min())
    return standardized

In [14]:
all_files = list(root.glob("la*"))
len(all_files)

20

In [15]:
save_root = Path('Preprocessed')

for counter, path_to_mri_data in enumerate(tqdm(all_files)):
    path_to_label = change_img_to_label_path(path_to_mri_data)
    mri = nib.load(path_to_mri_data)
    assert nib.aff2axcodes(data.affine) == ('R', 'A', 'S')
    mri_data = mri.get_fdata()
    label_data = nib.load(path_to_label).get_fdata().astype(np.uint8)
    
    mri_data = mri_data[32:-32, 32:-32]
    label_data = label_data[32:-32, 32:-32]
    
    normalized_mri_data = normalize(mri_data)
    standardized_mri_data = standardize(normalized_mri_data)
    
    if counter < 17:
        current_path = save_root/"train"/str(counter)
    else:
        current_path = save_root/"val"/str(counter)

  0%|          | 0/20 [00:00<?, ?it/s]